In [ ]:
# import tensorflow
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import mglearn
from sklearn.svm import SVC
from sklearn import metrics
from skimage.transform import resize
import random
import string

In [ ]:
data_test = np.load('./data_youtube/youtube_faces_with_keypoints_full_1/youtube_faces_with_keypoints_full_1/Aaron_Eckhart_0.npz')
print(data_test.files)
print(data_test['colorImages'].shape)
print(data_test['boundingBox'].shape)
print(data_test['landmarks2D'].shape)
print(data_test['landmarks3D'].shape)

x = data_test['colorImages']
if x.shape[2] == 3:
    x = np.rollaxis(x,3) # roll the axis to make it (231, 237, 3, 79) instead of (79, 231, 237, 3)
print(x.shape)
plt.imshow(x[78])

In [ ]:
path = './data_youtube/youtube_faces_with_keypoints_full_1/youtube_faces_with_keypoints_full_1/'

def save_image(images,labels):
    path = f'./data_youtube/data_images/{labels}'
    try:
        os.mkdir(path)
    except:
        pass
    for i in range(images.shape[0]):
        name = "".join(random.choices(string.ascii_uppercase + string.digits, k=10))
        # print(images[i])
        plt.imsave(f'{path}/{name}.png', images[i])
        

def load_data(path):
    cnt = 0
    print(len(os.listdir(path)))
    data = np.zeros((0, 244, 244, 3))
    labels = np.zeros((0, 1))
    name_old = ''
    for file in os.listdir(path):
        name = file.split('_')[0]
        if file.endswith('.npz'):
            data_test = np.load(path + file)
            x = data_test['colorImages']
            if x.shape[2] == 3:
                x = np.rollaxis(x,3) # roll the axis to make it (231, 237, 3, 79) instead of (79, 231, 237, 3)
            elif data_test["colorImages"].shape[1] == 3:
                x = np.rollaxis(data_test["colorImages"], 2)
            elif data_test["colorImages"].shape[0] == 3:
                x = np.rollaxis(data_test["colorImages"], 1)
            
            print("x shape is ", x.shape)
            x = resize(x, (x.shape[0],x.shape[1] * 244/(max(x.shape[1],x.shape[2])),x.shape[2] * 244/(max(x.shape[1],x.shape[2]))))
            # print("x shape is ", x.shape)
            x = x[:min(20, x.shape[0]),:,:,:] # take the first 200 frames
            # print("x shape is ", x.shape)
            x_train = np.zeros((x.shape[0], 244, 244, 3))
            x_train[:, :x.shape[1], :x.shape[2], :] = x
            # print("x_train shape is ", x_train.shape)
            data = np.append(data, x_train)
            data = data.reshape(-1, 244, 244, 3)
            # print("data shape is ", data.shape)
            save_image(x , name)
            if name != name_old:
                cnt += 1
                print(cnt)
                name_old = name
                
            labels = np.append(labels,np.full((x_train.shape[0], 1), cnt))
            if cnt == 3:
                break
    return data,labels

data,labels = load_data(path)
plt.imshow(data[40])
# save_image(data , "name")